<summary>Table of Contents</summary>

- [1. Pretrain](#1-pretrain)
- [2. Finetune](#2-finetune)


This notebook demonstrates experiments on 5 countries with self-supervised learning. 
All parameters and settings are taken from the paper.

In [1]:
import pandas as pd
import subprocess
import os
import time
from utils.helper import extract_metrics_from_output, running_time

# 1. Pretrain 

Tracking loss - MAE.

In [4]:
# Dynamic variables

countries = ['DE', 'GB', 'ES', 'FR', 'IT']
seq_lens = [512, 512, 336, 168, 168]
pred_lens = ["24", "96", "168"]

In [5]:
#cuda_device = "1"
#os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

start = time.time()

countries = ['DE', 'GB', 'ES', 'FR', 'IT']
#seq_lens = [512, 512, 336, 168, 168]
pred_lens = ["24", "96", "168"]

for i, country in enumerate(countries):
    for pred_len in pred_lens:
        if country == 'DE' and pred_len == '24':
            seq_len = 336
        else:
            seq_len = seq_lens[i]
            
        # Just use the same parameters as from the paper
        params = {
            "--context_points": seq_len,
            "--dset": country,
            "--mask_ratio": 0.4,
            "--scaler_type": "minmax",
            "--n_epochs_pretrain": 100,
            "--target_points": pred_len,
            "--batch_size": 128
        }

        # Build the command string
        command = "python PatchTST-main/PatchTST_self_supervised/patchtst_pretrain.py "

        # Add parameters to the command
        for key, value in params.items():
            if value is not None:
                command += f"{key} {value} "
            else:
                command += f"{key} "  # Add flags with no value

        # Execute the command
        print(f"==========Running command for {country}, pred_len {pred_len}:==========")
        !{command}
        
end = time.time()
hours, mins, secs = running_time(start, end)
print("Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs))

==========Running command for DE, pred_len 24:==========
args: Namespace(dset_pretrain='DE', context_points=336, target_points=24, batch_size=128, num_workers=0, features='M', patch_len=12, stride=12, revin=1, overlapping_windows=True, scaler_type='minmax', if_relu=False, n_layers=3, n_heads=16, d_model=128, d_ff=512, dropout=0.2, head_dropout=0.2, mask_ratio=0.4, n_epochs_pretrain=100, lr=0.0001, pretrained_model_id=1, model_type='based_model')
scaler_type: minmax overlapping_windows: True
number of patches: 28
number of model params 601612
suggested_lr 0.00020565123083486514
scaler_type: minmax overlapping_windows: True
number of patches: 28
number of model params 601612
          epoch     train_loss     valid_loss           time
Better model found at epoch 0 with valid_loss value: 0.9297727941028631.
              0       0.995705       0.929773          00:04
Better model found at epoch 1 with valid_loss value: 0.9217871140873813.
              1       0.948826       0.921787     

# 2. Finetune

Linear probing + full fine-tuning.

In [6]:
# Dynamic variables
#cuda_device = "1"
#os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

#countries = ['DE', 'GB', 'ES', 'FR', 'IT']
#seq_lens = [512, 512, 336, 168, 168]
#pred_lens = ["24", "96", "168"]

start = time.time()

# List to store metrics for the DataFrame
data = []

for i, country in enumerate(countries):
    for pred_len in pred_lens:
        if country == 'DE' and pred_len == '24':
            seq_len = 336
        else:
            seq_len = seq_lens[i]

        # Define the parameters for each country and pred_len
        params = {
            "--dset": country,
            "--context_points": seq_len,
            "--is_finetune": 1,
            "--pretrained_model": f"saved_models/{country}/masked_patchtst/based_model/patchtst_pretrained_cw{seq_len}_patch12_stride12_epochs-pretrain100_mask0.4_model1.pth",
            "--scaler_type": "minmax",
            "--n_epochs_finetune": 20,
            "--target_points": pred_len,
            "--batch_size": 128
        }

        # Build the command string
        command = "python PatchTST-main/PatchTST_self_supervised/patchtst_finetune.py "

        # Add parameters to the command
        for key, value in params.items():
            if value is not None:
                command += f"{key} {value} "
            else:
                command += f"{key} "  # Add flags with no value

        print(f"==========Running command for {country}, pred_len {pred_len}:==========")

        # Run the command and capture the output
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

        # Capture the output in real-time
        output = []
        for line in process.stdout:
            output.append(line)
            print(line, end='')  # Print in the .ipynb cell

        # Wait for the process to complete
        process.stdout.close()
        process.wait()

        try:
            # Extract metrics from the output using the function
            metrics = extract_metrics_from_output(output, if_supervised=False)
            mse, rmse, mae = metrics[0]

            # Append the results to the data list
            data.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae
            })
        except ValueError as e:
            print(f"\nError: {e} for {country}, pred_len {pred_len}")

end = time.time()
hours, mins, secs = running_time(start, end)
print("Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs))

==========Running command for DE, pred_len 24:==========
args: Namespace(is_finetune=1, is_linear_probe=0, dset_finetune='DE', context_points=336, target_points=24, batch_size=128, num_workers=0, scaler='standard', features='M', patch_len=12, stride=12, revin=1, overlapping_windows=True, scaler_type='minmax', if_relu=False, n_layers=3, n_heads=16, d_model=128, d_ff=256, dropout=0.2, head_dropout=0.2, n_epochs_finetune=20, lr=0.0001, pretrained_model='saved_models/DE/masked_patchtst/based_model/patchtst_pretrained_cw336_patch12_stride12_epochs-pretrain100_mask0.4_model1.pth', finetuned_model_id=1, model_type='based_model')
scaler_type: minmax overlapping_windows: True
number of patches: 28
number of model params 488728
check unmatched_layers: ['backbone.encoder.layers.0.ff.0.weight', 'backbone.encoder.layers.0.ff.0.bias', 'backbone.encoder.layers.0.ff.3.weight', 'backbone.encoder.layers.1.ff.0.weight', 'backbone.encoder.layers.1.ff.0.bias', 'backbone.encoder.layers.1.ff.3.weight', 'back

In [9]:
df_self_sup = pd.DataFrame(data)
df_self_sup.set_index(['Country', 'Pred_len'], inplace=True)
df_self_sup.columns = pd.MultiIndex.from_product([['Self-supervised'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])

# Path if not exists
path = "results/patchtst/"
if not os.path.exists(path):
    os.makedirs(path)

# Save the results to a CSV file
df_self_sup.to_csv(os.path.join(path, "finetune_results_128.csv"), index=True)
df_self_sup.round(4)

Model            Self-supervised                
Metrics                      MSE    RMSE     MAE
Country Pred_len                                
DE      24                0.0212  0.1456  0.0886
        96                0.0354  0.1880  0.1258
        168               0.0386  0.1966  0.1324
GB      24                0.0249  0.1578  0.1004
        96                0.0407  0.2018  0.1380
        168               0.0431  0.2077  0.1447
ES      24                0.0095  0.0976  0.0589
        96                0.0185  0.1361  0.0856
        168               0.0209  0.1447  0.0923
FR      24                0.0101  0.1003  0.0542
        96                0.0192  0.1387  0.0788
        168               0.0216  0.1470  0.0854
IT      24                0.0101  0.1004  0.0575
        96                0.0182  0.1350  0.0810
        168               0.0197  0.1403  0.0853

In [ ]:
# Только Франция 24 лучше выше...
df_self_sup = pd.DataFrame(data)
df_self_sup.set_index(['Country', 'Pred_len'], inplace=True)
df_self_sup.columns = pd.MultiIndex.from_product([['Self-supervised'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])

# Path if not exists
path = "results/patchtst/"
if not os.path.exists(path):
    os.makedirs(path)

# Save the results to a CSV file
df_self_sup.to_csv(os.path.join(path, "finetune_results.csv"), index=True)
df_self_sup.round(4)

Model            Self-supervised                
Metrics                      MSE    RMSE     MAE
Country Pred_len                                
DE      24                0.0210  0.1448  0.0878
        96                0.0353  0.1878  0.1256
        168               0.0379  0.1947  0.1318
GB      24                0.0247  0.1571  0.1001
        96                0.0409  0.2021  0.1390
        168               0.0429  0.2071  0.1445
ES      24                0.0094  0.0970  0.0581
        96                0.0185  0.1359  0.0867
        168               0.0208  0.1442  0.0924
FR      24                0.0100  0.1002  0.0537
        96                0.0193  0.1390  0.0785
        168               0.0212  0.1455  0.0856
IT      24                0.0101  0.1003  0.0571
        96                0.0181  0.1344  0.0795
        168               0.0197  0.1403  0.0853